In [1]:
import numpy as np
from keras.activations import softmax
def self(input_sequence):
    output = np.zeros(input_sequence.shape)
    for i, pivot_vector in enumerate(input_sequence):
        scores = np.zeros(shape=len(input_sequence))
        for j, vector in enumerate(input_sequence):
            scores[j] = np.dot(pivot_vector, vector.T)
        scores /= np.sqrt(input_sequence.shape[1])
        scores = softmax(scores)


In [2]:
import tensorflow as tf
from keras import layers
max_tokens = 20000
max_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32



class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([
            layers.Dense(dense_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim':self.embed_dim,
            'num_heads':self.num_heads,
            'dense_dim':self.dense_dim
        })
        return config

In [4]:
def layer_normalization(batch_of_sequences):
    mean = np.mean(batch_of_sequences, keepdims=True, axis=-1)
    variance = np.var(batch_of_sequences, keepdims=True, axis=-1)
    return (batch_of_sequences - mean) / variance

In [5]:
def batch_normalization(batch_of_images):
    mean = np.mean(batch_of_images, keepdims=True, axis=(0, 1, 2))
    variance = np.var(batch_of_images, keepdims=True, axis=(0, 1, 2))
    return (batch_of_images - mean) / variance

In [21]:
train_dir = 'aclImdb/train'
validation_dir = 'aclImdb/val'
test_dir = 'aclImdb/test'
batch_size = 32

train_ds = tf.keras.utils.text_dataset_from_directory(
    directory=train_dir,
    batch_size=batch_size
)
validation_ds = tf.keras.utils.text_dataset_from_directory(
    directory=validation_dir,
    batch_size=batch_size
)
test_ds = tf.keras.utils.text_dataset_from_directory(
    directory=test_dir,
    batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [43]:
from keras.layers import TextVectorization, Embedding
train_ds_text_only = train_ds.map(lambda x, y:x)
text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_length
)
text_vectorization.adapt(train_ds_text_only)
int_train_ds = train_ds.map(lambda x, y:(text_vectorization(x), y))
int_validation_ds = validation_ds.map(lambda x, y:(text_vectorization(x), y))

In [32]:
import keras

inputs = layers.Input(shape=(None,), dtype='int64')
# x = text_vectorization(inputs)
x = Embedding(input_dim=max_tokens, input_length=max_length, output_dim=embed_dim)(inputs)
x = TransformerEncoder(embed_dim,dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = keras.models.Model(inputs, outputs)

model_1.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

callbacks = [tf.keras.callbacks.ModelCheckpoint('Transformer model - text classification models/model_1_transformer_encoder',
                                                save_best_only=True)]

history_model_1 = model_1.fit(
    int_train_ds,
    epochs=1,
    validation_data=int_validation_ds,
    callbacks=callbacks
)

625/625 [==============================] - ETA: 0s - loss: 0.4952 - accuracy: 0.7692

INFO:tensorflow:Assets written to: Transformer model - text classification models\model_1_transformer_encoder\assets


INFO:tensorflow:Assets written to: Transformer model - text classification models\model_1_transformer_encoder\assets


625/625 [==============================] - 472s 754ms/step - loss: 0.4952 - accuracy: 0.7692 - val_loss: 0.3612 - val_accuracy: 0.8378


In [34]:
model_1.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_12 (Embedding)    (None, None, 256)         5120000   
                                                                 
 transformer_encoder_12 (Tra  (None, None, 256)        543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_3 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_29 (Dense)            (None, 1)                 257 

In [3]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim,**kwargs):
        super().__init__(**kwargs)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)


    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        position = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_position = self.position_embeddings(position)
        return embedded_tokens + embedded_position

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            'output_dim': self.output_dim,
            'sequence_length': self.sequence_length,
            'input_dim':self.input_dim
        })
        return config

In [5]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32
import keras
inputs = layers.Input(shape=(None,), dtype='int64')
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim,dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = keras.models.Model(inputs, outputs)


model_2.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

callbacks = [tf.keras.callbacks.ModelCheckpoint(
    'Transformer model - text classification models/model_2_transformer_encoder_positional_embeddings',
    save_best_only=True
)]

# history_model_2 = model_2.fit(
#     int_train_ds,
#     epochs=5,
#     validation_data=int_validation_ds,
#     callbacks=callbacks
# )

In [62]:
token = text_vectorization(["it is a glorious film, but you could not make it now. and that is not just my opinion my preview screening companion and fellow lover of weepy old black and white films agreed that not even the director max ophuls could get away with this 1948 classic if he tried to make it in 2009 but why not well for start it is visually sumptomps but in black and white if you remade it in clour you can never mathc But why not? Well, for a start, it is visually sumptuous, but in black and white. If you remade it in colour, you could never match the starkly beautiful composition of the shots, shy Lisa (Joan Fontaine) constantly framed in windows or peeping round doors, the dense black of monochrome night. The colour, the light, the texture of real life, would clutter up your eye and distract you from the lovely artifice. But why not? Well, for a start, it is visually sumptuous, but in black and white. If you remade it in colour, you could never match the starkly beautiful composition of the shots, shy Lisa (Joan Fontaine) constantly framed in windows or peeping round doors, the dense black of monochrome night. The colour, the light, the texture of real life, would clutter up your eye and distract you from the lovely artifice. "])
preds = model_2.predict(token)
print(f'{preds[0]} of positivity')

1/1 [==============================] - 0s 27ms/step
[0.9776884] of positivity


In [60]:
"it is a glorious film, but you could not make it now. and that is not just my opinion my preview screening companion and fellow lover of weepy old black and white films agreed that not even the director max ophuls could get away with this 1948 classic if he tried to make it in 2009 but why not well for start it is visually sumptomps but in black and white if you remade it in clour you can never mathc But why not? Well, for a start, it is visually sumptuous, but in black and white. If you remade it in colour, you could never match the starkly beautiful composition of the shots, shy Lisa (Joan Fontaine) constantly framed in windows or peeping round doors, the dense black of monochrome night. The colour, the light, the texture of real life, would clutter up your eye and distract you from the lovely artifice. But why not? Well, for a start, it is visually sumptuous, but in black and white. If you remade it in colour, you could never match the starkly beautiful composition of the shots, shy Lisa (Joan Fontaine) constantly framed in windows or peeping round doors, the dense black of monochrome night. The colour, the light, the texture of real life, would clutter up your eye and distract you from the lovely artifice. "

'it is a glorious film, but you could not make it now. and that is not just my opinion my preview screening companion and fellow lover of weepy old black and white films agreed that not even the director max ophuls could get away with this 1948 classic if he tried to make it in 2009'

In [ ]:
model_2.summary()